In [162]:
import pdal
import numpy as np

In [163]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


In [164]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 5
    }
  ]
}"""

In [165]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
pts = r.arrays[0]

In [166]:
pts

array([(333499.29, 7395564.35, 735.34, 18, 1, 1, 0, 0,  6, -14., 17, 6, 356569.36606547),
       (333499.1 , 7395569.68, 735.3 , 17, 1, 1, 0, 0,  6, -15., 17, 6, 356569.36615345),
       (333499.29, 7395521.99, 751.69, 10, 1, 1, 0, 0,  6, -11., 17, 6, 356569.38359445),
       ...,
       (333520.06, 7395558.13, 729.85,  2, 1, 2, 1, 0, 19,  13., 40, 6, 363044.03373891),
       (333587.32, 7395563.87, 744.67, 14, 1, 1, 1, 0,  6,   7., 40, 6, 363044.06873989),
       (333563.24, 7395571.79, 729.03,  5, 2, 2, 0, 0,  6,   9., 40, 6, 363044.3104589 )],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8')])

In [167]:
pt_edif = pts['Classification'] == 6
len(pts[pt_edif])

18835

In [168]:
pt_ground = pts['Classification'] == 2
len(pts[pt_ground])

983

In [169]:
len(pts[pt_ground | pt_edif])

19818

In [232]:
# Escolhendo um ponto aleatório no Ground

pt = np.random.choice(pts[pt_ground])

In [233]:
coord = ['X', 'Y', 'Z']

In [234]:
pt[coord]

(333413.2, 7395008.12, 726.62)

In [235]:
# Centralizando a visão em 1.5 metros acima do solo do ponto escolhido

pts_0 = np.array([pts[pt_ground | pt_edif]['X'] - pt['X'],
         pts[pt_ground | pt_edif]['Y'] - pt['Y'],
         pts[pt_ground | pt_edif]['Z'] - (pt['Z'] + 1.5)]).T

In [236]:
pts_0 = pts_0[pts_0[:, 2] >= 0.0]

In [237]:
# Calcluando Azimute, ou melhor, um angulo começando Leste 
# crescendo no sentido anti-horario e negativo para todo Y < 0

teste_az = [[0., 1],
           [1., 0.],
           [0., -1.],
           [-1., 0.]]

teste_az = np.array(teste_az)

az = np.arctan2(teste_az[:, 1], teste_az[:, 0]) * 180 / np.pi
az

array([ 90.,   0., -90., 180.])

In [238]:
# Calculando o angulo de orientação de cada ponto

orient = np.arctan2(pts_0[:, 1], pts_0[:, 0]) * 180 / np.pi
# orient

In [239]:
# Calculando o angulo de altura de cada ponto

# np.arctan2(10, 1000) * 180 / np.pi
# np.sqrt(np.square(4) + np.square(3))

inclin = np.arctan2(pts_0[:, 2], np.sqrt(np.square(pts_0[:, 0]) + np.square(pts_0[:, 1]))) * 180 / np.pi

In [240]:
# sorted(inclin, reverse=True)

In [241]:
orient


array([81.20190666, 81.30306617, 80.48940871, ..., 75.26820266,
       72.60389532, 75.09439854])

In [310]:
# Separando em fases

qt_fases = 60
fases = np.arange(-180., 180., 360./qt_fases)

pt_bins = np.digitize(orient, fases)

In [266]:
np.histogram(orient, fases)

(array([ 552,  153,    7,    2,    1,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   35,   82,   73,   87,   49,   27,
          33,   69,   16,   13,    0,    1,    7,  265,  412,  841,  746,
        1089,  748, 1127, 1082, 1344, 1343, 1417,  755,  802, 1069, 1161,
         972, 1145,  744,  548]),
 array([-180., -174., -168., -162., -156., -150., -144., -138., -132.,
        -126., -120., -114., -108., -102.,  -96.,  -90.,  -84.,  -78.,
         -72.,  -66.,  -60.,  -54.,  -48.,  -42.,  -36.,  -30.,  -24.,
         -18.,  -12.,   -6.,    0.,    6.,   12.,   18.,   24.,   30.,
          36.,   42.,   48.,   54.,   60.,   66.,   72.,   78.,   84.,
          90.,   96.,  102.,  108.,  114.,  120.,  126.,  132.,  138.,
         144.,  150.,  156.,  162.,  168.,  174.]))

In [313]:
np.unique(pt_bins)

array([ 1,  2,  3,  4,  5, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
       58, 59, 60])

In [314]:
np.arange(1, qt_fases + 1)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60])

In [315]:
list(map(lambda x: [x, np.nanmax(inclin[pt_bins == x])] if len(inclin[pt_bins == x]) > 0 else [x, 0.], np.arange(1, qt_fases + 1)))
# list(map(lambda x: len(orient[pt_bins == x]), fases))
# np.nanmax(inclin[pt_bins == 3]) if len(inclin[pt_bins == 3]) > 0

[[1, 27.868341628413496],
 [2, 30.788736481309538],
 [3, 9.000224766748621],
 [4, 5.315984535693404],
 [5, 2.580021660032749],
 [6, 0.0],
 [7, 0.0],
 [8, 0.0],
 [9, 0.0],
 [10, 0.0],
 [11, 0.0],
 [12, 0.0],
 [13, 0.0],
 [14, 0.0],
 [15, 0.0],
 [16, 0.0],
 [17, 0.0],
 [18, 0.0],
 [19, 0.0],
 [20, 0.0],
 [21, 0.0],
 [22, 0.0],
 [23, 0.0],
 [24, 0.0],
 [25, 0.0],
 [26, 0.0],
 [27, 0.0],
 [28, 42.09735579929636],
 [29, 43.45868586412693],
 [30, 44.718430420285344],
 [31, 43.371536196258134],
 [32, 11.55562907426191],
 [33, 11.438820029681969],
 [34, 7.282389601492088],
 [35, 8.366632884649908],
 [36, 9.100219579976116],
 [37, 6.0340936007551855],
 [38, 0.0],
 [39, 1.0476413422467141],
 [40, 1.935566626524035],
 [41, 5.586191505779991],
 [42, 6.424686464399003],
 [43, 8.175254345632721],
 [44, 6.51884059837617],
 [45, 8.868369121716592],
 [46, 10.10215427090378],
 [47, 11.097043474002499],
 [48, 16.368009992092002],
 [49, 37.604749226173396],
 [50, 39.82748847613857],
 [51, 42.7934092295428

array([ True,  True,  True, ..., False, False, False])